In [19]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

In [20]:
df = pd.read_csv('dataset/imdb_top_1000.csv')
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [21]:
df.shape

(1000, 16)

## Question
- Create new metadata with Genre, Overview, Director and Star 1-4
- Preprocessing the text with preprocessing part 1-3
- Create recommendation system with cosine distance
- Try the recommendation system is it good or bad ?

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
df.isna().sum()

Poster_Link        0
Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

In [24]:
df_m=df.copy()

In [25]:
df_m['Meta Data']=df['Genre']+df['Overview']+df['Director']+df['Star1']+df['Star2']+df['Star3']+df['Star4']
df_m.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Meta Data
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",DramaTwo imprisoned men bond over a number of ...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","Crime, DramaAn organized crime dynasty's aging..."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","Action, Crime, DramaWhen the menace known as t..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","Crime, DramaThe early life and career of Vito ..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","Crime, DramaA jury holdout attempts to prevent..."


In [26]:
contractions_dict = {    
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i had",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "iit will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they had",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

In [27]:
from string import punctuation
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer 

stop_words = stopwords.words('english')
stop_words.remove('not')
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()


def to_lower(text):
    return text.lower()

def remove_contraction(text):
    list_kata = text.split()
    list_hasil = []

    for kata in list_kata:

        if kata in contractions_dict.keys():
            list_hasil.append(contractions_dict[kata])
        else:
            list_hasil.append(kata)

    hasil = ' '.join(list_hasil)
    return hasil 

def remove_number(text):
    hasil = ''.join([char for char in text if not char.isnumeric()])
    return hasil

def remove_punctuation(text):
    hasil = ''.join([char for char in text if not char in punctuation])
    return hasil

def remove_stopwords(text):
    hasil = ' '.join([kata for kata in text.split() if kata not in stop_words])
    return hasil 

def remove_whitespace(text):
    hasil = ' '.join(text.split())
    return hasil

def stem(text):
    list_hasil = []

    for sentence in nltk.sent_tokenize(text):   # pecah paragraf menjadi kalimat-kalimat
        for word in nltk.word_tokenize(sentence): # pecah kalimat menjadi kata-kata
            list_hasil.append(stemmer.stem(word))   # ubah kata ke kata dasarnya

    hasil = ' '.join(list_hasil)
    return hasil 

def lemmatize(text):
    list_hasil = []

    for sentence in nltk.sent_tokenize(text):   # pecah paragraf menjadi kalimat-kalimat
        for word in nltk.word_tokenize(sentence): # pecah kalimat menjadi kata-kata
            list_hasil.append(lemmatizer.lemmatize(word))   # ubah kata ke kata dasarnya

    hasil = ' '.join(list_hasil)
    return hasil 


df_m['text-prep'] = df_m['Meta Data'].apply(to_lower)
df_m['text-prep'] = df_m['text-prep'].apply(remove_contraction)
df_m['text-prep'] = df_m['text-prep'].apply(remove_number)
df_m['text-prep'] = df_m['text-prep'].apply(remove_punctuation)
df_m['text-prep'] = df_m['text-prep'].apply(remove_stopwords)
df_m['text-prep'] = df_m['text-prep'].apply(remove_whitespace)
df_m['text-prep-lemm'] = df_m['text-prep'].apply(lemmatize)
df_m.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Meta Data,text-prep,text-prep-lemm
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",DramaTwo imprisoned men bond over a number of ...,dramatwo imprisoned men bond number years find...,dramatwo imprisoned men bond number year findi...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","Crime, DramaAn organized crime dynasty's aging...",crime dramaan organized crime dynastys aging p...,crime dramaan organized crime dynasty aging pa...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","Action, Crime, DramaWhen the menace known as t...",action crime dramawhen menace known joker wrea...,action crime dramawhen menace known joker wrea...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","Crime, DramaThe early life and career of Vito ...",crime dramathe early life career vito corleone...,crime dramathe early life career vito corleone...
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","Crime, DramaA jury holdout attempts to prevent...",crime dramaa jury holdout attempts prevent mis...,crime dramaa jury holdout attempt prevent misc...


In [28]:
df_meta=df_m[['Poster_Link','Series_Title','text-prep-lemm']]
df_meta.head()

,Poster_Link,Series_Title,text-prep-lemm
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,dramatwo imprisoned men bond number year findi...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,crime dramaan organized crime dynasty aging pa...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,action crime dramawhen menace known joker wrea...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,crime dramathe early life career vito corleone...
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,crime dramaa jury holdout attempt prevent misc...


In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tf = TfidfVectorizer(norm=None)
bow = tf.fit_transform(df_m['text-prep-lemm'])

In [30]:
df_meta=df_meta.reset_index().rename(columns={'index':'id','text-prep-lemm':'tag'})
df_meta.head()

,id,Poster_Link,Series_Title,tag
0,0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,dramatwo imprisoned men bond number year findi...
1,1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,crime dramaan organized crime dynasty aging pa...
2,2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,action crime dramawhen menace known joker wrea...
3,3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,crime dramathe early life career vito corleone...
4,4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,crime dramaa jury holdout attempt prevent misc...


In [31]:
df_meta.to_csv('imdb_top_meta.csv', index=False)

In [32]:
df_meta['Poster_Link']=df_m['Poster_Link']

In [33]:
from sklearn.metrics.pairwise import cosine_distances
idx = 0 #anggap sudah tonton toystory

content = df_meta.loc[idx,'tag']
watched = tf.transform([content])
dist = cosine_distances(watched,bow)
rec_idx = dist.argsort()[0,1:101]
df_meta.loc[rec_idx]

,id,Poster_Link,Series_Title,tag
680,680,https://m.media-amazon.com/images/M/MV5BM2ZmND...,Down by Law,comedy crime dramatwo men framed sent jail mee...
68,68,https://m.media-amazon.com/images/M/MV5BMTI3NT...,Oldeuboi,action drama mysteryafter kidnapped imprisoned...
179,179,https://m.media-amazon.com/images/M/MV5BNzA2Nm...,The Great Escape,adventure drama historyallied prisoner war pla...
683,683,https://m.media-amazon.com/images/M/MV5BOTM5N2...,The Breakfast Club,comedy dramafive high school student meet satu...
350,350,https://m.media-amazon.com/images/M/MV5BYjU1Nj...,Dev.D,drama romanceafter breaking childhood sweethea...
...,...,...,...,...
927,927,https://m.media-amazon.com/images/M/MV5BNzU3ND...,Harry Potter and the Half-Blood Prince,action adventure familyas harry potter begin s...
506,506,https://m.media-amazon.com/images/M/MV5BMTY4NT...,Harry Potter and the Prisoner of Azkaban,adventure family fantasyharry potter ron hermi...
410,410,https://m.media-amazon.com/images/M/MV5BYTViNz...,The Terminator,action scifia human soldier sent stop almost i...
960,960,https://m.media-amazon.com/images/M/MV5BMzk1Mm...,Sleepers,crime drama thrillerafter prank go disastrousl...


In [34]:
import pickle

pickle.dump(tf,open('tfidf.pkl','wb'))
pickle.dump(bow,open('similarity.pkl','wb'))